In [8]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import os, json
from pyspark.sql.functions import *

spark = (
    SparkSession.builder.appName("ExerciseSpark")
    .getOrCreate()
)

In [94]:
path_to_json = '../datalake/bronze/lofi-hiphop/'
json_files = [  '../datalake/bronze/lofi-hiphop/' + pos_json for pos_json in os.listdir(path_to_json) 
              if pos_json.endswith('.json')]

df = (
    spark
    .read
    .format("json")
    .load(json_files, multiLine=True)
)

In [95]:
cols = ["author", "messageEx"]
df = (
    df
    .withColumn("badgeUrl", col("author.badgeUrl"))
    .withColumn("channelId", col("author.channelId"))
    .withColumn("channelUrl", col("author.channelUrl"))
    .withColumn("imageUrl", col("author.imageUrl"))
    .withColumn("isChatModerator", col("author.isChatModerator"))
    .withColumn("isChatOwner", col("author.isChatOwner"))
    .withColumn("isChatSponsor", col("author.isChatSponsor"))
    .withColumn("isVerified", col("author.isVerified"))
    .withColumn("name", col("author.name"))
    .withColumn("type", col("author.type"))
    .withColumn("year", year(col("datetime")))
    .withColumn("month", month(col("datetime")))
    .drop(*cols)

)

In [97]:
df.printSchema()

root
 |-- amountString: string (nullable = true)
 |-- amountValue: double (nullable = true)
 |-- bgColor: long (nullable = true)
 |-- currency: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- elapsedTime: string (nullable = true)
 |-- id: string (nullable = true)
 |-- message: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- type: string (nullable = true)
 |-- badgeUrl: string (nullable = true)
 |-- channelId: string (nullable = true)
 |-- channelUrl: string (nullable = true)
 |-- imageUrl: string (nullable = true)
 |-- isChatModerator: boolean (nullable = true)
 |-- isChatOwner: boolean (nullable = true)
 |-- isChatSponsor: boolean (nullable = true)
 |-- isVerified: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



In [67]:
df_pandas = df.toPandas()

In [102]:
df_pandas.to_parquet(
    '../datalake/silver/lofi-hiphop/data.parquet',
    partition_cols=['year', 'month'])